In [2]:
import pandas as pd
from uszipcode import SearchEngine
from meteostat import Point, Hourly
from datetime import datetime
import numpy as np

In [15]:
def find_zip(lat, lon):
    zip_df = pd.read_csv('../ref/US_long_lat_to_zip.csv')
    best_zip = None
    best_zip_distance = None
    for index, row in zip_df.iterrows():
        x1 = float(row['LAT'])
        y1 = float(row['LNG'])
        x_dif = (lat - x1)**2
        y_diff = (lon - y1)**2
        distance = np.sqrt(x_dif + y_diff)
        if not best_zip_distance or distance < best_zip_distance:
            best_zip_distance = distance
            best_zip = row['ZIP']
    return best_zip

In [4]:
def search_by_zip(zip_code):
    search = SearchEngine(simple_zipcode=True)  # set simple_zipcode=False to use rich info database
    zipcode = search.by_zipcode(zip_code)
    return zipcode


In [26]:
def search_weather(lat, lon):
    start = datetime(2021, 1, 20)
    end = datetime(2021, 1, 21)

    # Create Point for Vancouver, BC
    location = Point(lat, lon, 70)

    # Get daily data for 2021
    hour_df = Hourly(location, start, end)
    hour_df = hour_df.fetch()

    return hour_df

In [31]:
def find_additional_info(lat, lon):
    out_dict = {}
    now = datetime.now()

    zipcode = int(find_zip(lat, lon))
    print("Zipcode", zipcode)
    population_info = search_by_zip(zipcode).to_dict()
    print("population info", population_info)
    weather_info = search_weather(lat, lon)
    print("weather_info", weather_info)
    hour_search = (now.hour - 3)
    if (now.hour - 3) < 0:
        hour_search = 0
    elif (now.hour - 3) > 23:
        hour_search = 23
    print("Hour search", hour_search)
    hour_row = weather_info.iloc[(hour_search),:].to_dict()

    for key in population_info.keys():
        val = population_info[key]
        if isinstance(val, list):
            for list_index, list_val in enumerate(val):
                if list_index == 0:
                    out_dict[key] = list_val
                else:
                    list_key = key + str(list_index)
                    out_dict[list_key] = list_val
            print(key, "Is List")
        else:
            out_dict[key] = val


    for item in hour_row.keys():
        out_dict[item] = hour_row[item]

    return out_dict


In [32]:
lat, lon = 49.2497, -123.1193
additional_info = find_additional_info(lat, lon)

Zipcode 98281
population info {'zipcode': '98281', 'zipcode_type': 'Standard', 'major_city': 'Point Roberts', 'post_office_city': 'Point Roberts, WA', 'common_city_list': ['Point Roberts'], 'county': 'Whatcom County', 'state': 'WA', 'lat': 48.99, 'lng': -123.06, 'timezone': 'Pacific', 'radius_in_miles': 2.0, 'area_code_list': ['360'], 'population': 1314, 'population_density': 269.0, 'land_area_in_sqmi': 4.89, 'water_area_in_sqmi': 1.69, 'housing_units': 2068, 'occupied_housing_units': 678, 'median_home_value': 221800, 'median_household_income': 39500, 'bounds_west': -123.090546, 'bounds_east': -123.004856, 'bounds_north': 49.002154, 'bounds_south': 48.971295}
weather_info                      temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  \
time                                                                           
2021-01-20 00:00:00   6.9   2.9  76.0   0.0   NaN    0.0   0.0   NaN  1027.2   
2021-01-20 01:00:00   6.7   3.5  80.0   0.0   NaN  118.0   7.9   NaN  1026.8   

In [33]:
print(additional_info)

{'temp': 4.9, 'dwpt': 3.2, 'rhum': 89.0, 'prcp': 0.0, 'snow': nan, 'wdir': 329.0, 'wspd': 3.2, 'wpgt': nan, 'pres': 1022.3, 'tsun': nan, 'coco': 4.0, 'zipcode': '98281', 'zipcode_type': 'Standard', 'major_city': 'Point Roberts', 'post_office_city': 'Point Roberts, WA', 'common_city_list': 'Point Roberts', 'county': 'Whatcom County', 'state': 'WA', 'lat': 48.99, 'lng': -123.06, 'timezone': 'Pacific', 'radius_in_miles': 2.0, 'area_code_list': '360', 'population': 1314, 'population_density': 269.0, 'land_area_in_sqmi': 4.89, 'water_area_in_sqmi': 1.69, 'housing_units': 2068, 'occupied_housing_units': 678, 'median_home_value': 221800, 'median_household_income': 39500, 'bounds_west': -123.090546, 'bounds_east': -123.004856, 'bounds_north': 49.002154, 'bounds_south': 48.971295}
